<a href="https://colab.research.google.com/github/sgu20191619/DACON/blob/main/LGAimers_%EC%98%A4%ED%94%84%EB%9D%BC%EC%9D%B8%ED%95%B4%EC%BB%A4%ED%86%A4_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D%EC%B4%88%EB%B3%B4%EC%9E%90(%EC%9D%B4%EB%8F%99%EC%84%9D%2C_%EC%A0%95%EC%84%9C%EC%9C%A4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 모듈 설치 
!pip install --quiet xgboost
!pip install --quiet catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 9.2 MB/s eta 0:00:00


In [ ]:
import platform
platform.platform()

'Linux-5.10.147+-x86_64-with-glibc2.31'

In [ ]:
!cat /etc/issue.net

Ubuntu 20.04.5 LTS


In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from collections import Counter
from scipy.stats import pearsonr

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import accuracy_score
from collections import OrderedDict
import random
import os

In [ ]:
!python --version
print("Skleran Version :", sklearn.__version__)
print('Pandas Version :', pd.__version__)
print('Numpy Version :', np.__version__)

Python 3.9.16
Skleran Version : 1.2.2
Pandas Version : 1.4.4
Numpy Version : 1.22.4


In [ ]:
# 이상치 처리
def clean_column(col, reference_col=None):
    if reference_col is None:
        reference_col = col
        
    z_scores = (reference_col - reference_col.mean()) / reference_col.std()
    col_cleaned = col.mask(abs(z_scores) > 3)

    most_frequent_value = reference_col.mode()[0]

    col_filled = col_cleaned.fillna(most_frequent_value)

    return col_filled

In [ ]:
# 스태킹 앙상블을 진행하고 최종적으로 예측까지 진행하는 함수 
def stacking_and_pred(final_X, new_test) :
  # 최종 학습시킬 train데이터 fianl_X 나눔 
  X_train, X_val, y_train, y_val = train_test_split(final_X, y_class, test_size=0.2, random_state=37)

  # 가장 보편적으로 많이 사용하는 5가지 모델을 베이스 모델로 설정 
  base_models = [
      GradientBoostingClassifier(random_state=37),
      RandomForestClassifier(n_estimators=2000, random_state=37),
      xgb.XGBClassifier(random_state = 37),
      LGBMClassifier(random_state = 37),
      CatBoostClassifier(random_state=37, verbose = 0), 
  ]
  
  # 메타 모델로 xgboost 선정 
  meta_model = xgb.XGBClassifier(random_state = 37)

  # K_Fold를 통한 스태킹 앙상블 진행
  n_splits = 5
  kf = KFold(n_splits=n_splits, shuffle=True, random_state=37)

  X_train_meta = np.zeros((X_train.shape[0], len(base_models)))

  # validation으로 메타모델의 f1_score 검증
  for i, model in enumerate(base_models):
      for train_idx, val_idx in kf.split(X_train):
          X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
          y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

          model.fit(X_train_fold, y_train_fold)

          y_val_pred = model.predict(X_val_fold)

          X_train_meta[val_idx, i] = y_val_pred.ravel()

  meta_model.fit(X_train_meta, y_train)

  X_val_meta = np.zeros((X_val.shape[0], len(base_models)))
  for i, model in enumerate(base_models):
      model.fit(X_train, y_train)

      y_val_pred = model.predict(X_val)
      X_val_meta[:, i] = y_val_pred.ravel()

  y_pred = meta_model.predict(X_val_meta)

  accuracy = f1_score(y_val, y_pred, average='macro')
  print("F1 score:", accuracy)

  # 각 베이스 모델의 test데이터 예측 결과를 담을 변수 
  X_test_meta = np.zeros((new_test.shape[0], len(base_models)))

  # 전체 데이터로 학습을 시킨 후 예측한 후 X_test_meta에 값 저장 
  for i, model in enumerate(base_models):
      model.fit(final_X, y_class)
      y_test_pred = model.predict(new_test)
      X_test_meta[:, i] = y_test_pred.ravel()

  # 메타 모델로 최종 y_pred 예측 
  y_pred = meta_model.predict(X_test_meta)
  # 이후 결과 및 f1_socre 반환 
  return y_pred, accuracy

In [ ]:
# 가장 좋은 성능을 보여줄 feature 개수를 찾기 위한 함수 
def find(X, bound, new_test) : 
  # Y_Class와 Y_Quality와 나머지 feature들의 pearsons 상관계수를 활용 해 feature_selection 진행 
  correlations = []
  for feature in X.columns:
    correlation, _ = pearsonr(X[feature], y_class)
    correlations.append(correlation)

  correlations2 = []
  for feature in X.columns:
      correlation, _ = pearsonr(X[feature], y_quality)
      correlations2.append(correlation)

  k = len(X.columns) // 3
  top_k_indices = sorted(range(len(correlations)), key=lambda i: abs(correlations[i]), reverse=True)[:k]
  top_k_features = X.columns[top_k_indices]

  top_k_indices = sorted(range(len(correlations2)), key=lambda i: abs(correlations2[i]), reverse=True)[:k]
  top_k_features2 = X.columns[top_k_indices]

  top_features = list(OrderedDict.fromkeys(list(top_k_features) + list(top_k_features2)))
  new_X = X[top_features]

  # 위 new_X를 train으로 여러 모델을 돌려 본 결과 validation이 RandomForest가 가장 높았음 
  model = RandomForestClassifier(random_state=37)
  model.fit(new_X, y_class)

  # 학습 후, feature_importance로 한번 더 feature selection 진행 
  importances = model.feature_importances_
  feature_importances = pd.DataFrame({'feature': new_X.columns, 'importance': importances})
  feature_importances = feature_importances.sort_values('importance', ascending = False)

  # feature_importance에서 150~300개를 선택하여 가장 높은 f1_score를 기록한 개수 및 결과값을 저장
  for j in bound : 
    new_cols = feature_importances[:j]['feature']
  
    print("현재 {} 의 feature_importance로 선택된 컬럼 수는 : {}".format(model, len(new_cols)))

    final_X = new_X[new_cols]
    new_test = new_test[new_cols]
    y_pred, accuracy = stacking_and_pred(final_X, new_test)
  
    file_name = f"prepro-t(rb)_cat_f1_score={accuracy: .4f},bound={j}.csv"
    submit["Y_Class"] = y_pred
    submit.to_csv("/content/drive/MyDrive/" +file_name, index = False)

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/test.csv")
submit = pd.read_csv("/content/drive/MyDrive/sample_submission.csv")

In [ ]:
train_df.shape

(1132, 3331)

In [ ]:
train_df.describe()

In [ ]:
# PRODUCT_ID 와 TIMESTAMP 드랍 
train_df = train_df.drop(columns=['PRODUCT_ID'])
test_df = test_df.drop(columns=['PRODUCT_ID'])

# 코드 공유에 나와있는 라벨 인코딩 
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_df[i])
    train_df[i] = le.transform(train_df[i])
    
    for label in np.unique(test_df[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_df[i] = le.transform(test_df[i]) 
print('Done.')

In [ ]:
# 모두 nan값인 column
all_nan_cols = train_df.columns[train_df.isna().all()].tolist()

print('Columns with all missing values:')
print(all_nan_cols)
print(len(all_nan_cols))

# 유일한 값이 하나만 존재하는 column
const_cols = train_df.columns[train_df.nunique() == 1].tolist()

print('Columns with only one unique value:')
print(const_cols)
print(len(const_cols))

train_df = train_df.drop(all_nan_cols, axis=1)
train_df = train_df.drop(const_cols, axis=1)

print(len(train_df.columns))
# *2drop -> 2794(나와야함)

In [ ]:
# train과 동일한 전처리 
test_df = test_df.drop(all_nan_cols, axis=1)
test_df = test_df.drop(const_cols, axis=1)

In [ ]:
# 전처리 
# Y_Class, LINE, PRODUCT_CODE 별로 나눈 후 평균 값으로 결측치를 대체하려 했으나, 데이터가 불균형하여 0으로 대체하기로 결정 

# Y_Class 별로 나눔 
train_class = []
for i in range(3) :
  train_class.append(train_df.loc[train_df["Y_Class"].isin([i])])

# Y_Class 별로 나눈 데이터에서 다시 LINE별로 나눔 
line = train_df["LINE"].unique()
train_class_line = []
for df in train_class :
  for l in line :
    train_class_line.append(df.loc[df["LINE"].isin([l])])

# Y_Class, LINE 별로 나눈 데이터에서 다시 PRODUCT_CODE로 나눔 
code = train_df["PRODUCT_CODE"].unique()
train_class_line_code = [] 
for df in train_class_line :
  for c in code :
    if (len(df.loc[df["PRODUCT_CODE"].isin([c])]) != 0) : 
      train_class_line_code.append(df.loc[df["PRODUCT_CODE"].isin([c])])

# 0으로 결측치 처리하기로 결정 
for i in range(len(train_class_line_code)) :
  train_class_line_code[i] =  train_class_line_code[i].fillna(0)

# 나눴던 데이터를 다시 합침
train = pd.concat(train_class_line_code)
train = train.reset_index(drop = True)

# Y_class와 Y_Quality에 target을 저장 후 드람
y_class = train["Y_Class"]
y_quality = train["Y_Quality"]
train = train.drop(columns=['Y_Class', 'Y_Quality'])

X = train 

test_df = test_df.fillna(0)

In [ ]:
# 이상치 처리 
X_t = X.apply(clean_column)
test_df = test_df.apply(lambda col: clean_column(col, reference_col=X[col.name]))

In [ ]:
# 이상치에 강한 로버스트 스케일링 
from sklearn.preprocessing import RobustScaler

rb = RobustScaler()
rb.fit(X_t)
train_scaled = rb.transform(X_t)
test_scaled = rb.transform(test_df)

X_train = pd.DataFrame(train_scaled)
X_test = pd.DataFrame(test_scaled)

In [ ]:
bound = range(208,209)

find(X_train, bound, X_test)